In [1]:
import imdb
from rdflib import Graph, URIRef, RDF, Literal, Namespace
from rdflib.namespace import RDF
import pandas as pd


In [2]:
homedir='/home/luitheob/AIFB/'
#pd.read_csv(homedir + 'data/IMDB/entity2id.txt', sep=" ", header=None)
movies = pd.read_table(homedir + 'data/IMDB/entity2id.txt', delim_whitespace=True)#, names=('A', 'B', 'C'))

In [3]:
ia = imdb.IMDb()
movie = ia.get_movie('0008827')
print(movie)

Amarilly of Clothes-Line Alley


In [19]:
print(movie['genres'])
print(movie['title'])
print(movie['cast'])
print(movie['director'])

['Comedy', 'Romance']
Amarilly of Clothes-Line Alley
[<Person id:0681933[http] name:_Mary Pickford_>, <Person id:0779914[http] name:_William Scott_>, <Person id:0696985[http] name:_Kate Price_>, <Person id:0914003[http] name:_Ida Waterman_>, <Person id:0449908[http] name:_Norman Kerry_>, <Person id:0329573[http] name:_Fred Goodwins_>, <Person id:0484838[http] name:_Margaret Landis_>, <Person id:0934232[http] name:_Tom Wilson_>, <Person id:0058191[http] name:_Wesley Barry_>, <Person id:0125331[http] name:_Frank Butterworth_>, <Person id:0352383[http] name:_George Hackathorne_>, <Person id:0543583[http] name:_Marcia Manon_>, <Person id:0795025[http] name:_Antrim Short_>, <Person id:0795053[http] name:_Gertrude Short_>, <Person id:0822059[http] name:_Herbert Standing_>, <Person id:0824885[http] name:_Larry Steers_>, <Person id:0167032[http] name:_Gustav von Seyffertitz_>, <Person id:0925118[http] name:_Leo White_>]
[<Person id:0624714[http] name:_Marshall Neilan_>]


In [29]:
movie['cast'][0].keys()

['name', 'canonical name', 'long imdb name', 'long imdb canonical name']

In [38]:
movie['cast']

[<Person id:0681933[http] name:_Mary Pickford_>,
 <Person id:0779914[http] name:_William Scott_>,
 <Person id:0696985[http] name:_Kate Price_>,
 <Person id:0914003[http] name:_Ida Waterman_>,
 <Person id:0449908[http] name:_Norman Kerry_>,
 <Person id:0329573[http] name:_Fred Goodwins_>,
 <Person id:0484838[http] name:_Margaret Landis_>,
 <Person id:0934232[http] name:_Tom Wilson_>,
 <Person id:0058191[http] name:_Wesley Barry_>,
 <Person id:0125331[http] name:_Frank Butterworth_>,
 <Person id:0352383[http] name:_George Hackathorne_>,
 <Person id:0543583[http] name:_Marcia Manon_>,
 <Person id:0795025[http] name:_Antrim Short_>,
 <Person id:0795053[http] name:_Gertrude Short_>,
 <Person id:0822059[http] name:_Herbert Standing_>,
 <Person id:0824885[http] name:_Larry Steers_>,
 <Person id:0167032[http] name:_Gustav von Seyffertitz_>,
 <Person id:0925118[http] name:_Leo White_>]

In [125]:
ex = Namespace('http://imdb.org/')
ont = Namespace('http://imdb.org/ontology/')
title_uri = URIRef(ont+'has_title') #raus
has_genre_uri = URIRef(ont+'has_genre')
director_uri = URIRef(ont+'has_director')
dir_mov_uri = URIRef(ont+'directed_by')
cast_uri = URIRef(ont+'has_actor')
cast_mov_uri = URIRef(ont+'acted_in')
published_uri = URIRef(ont+'published_in') #Jahr
collab_uri = URIRef(ont+'collabs_with')
prefers_genre = URIRef(ont+'preferred_by') #kritisch

dict = {}
test_train = []
rdf_graph = Graph()
# Genres to include
included_genres = ['Drama', 'Comedy', 'Romance', 'Thriller']
id = 0 
for movie_id in movies['movie'][:1500]:
    # Fetch movie information from IMDb
    try:
        movie = ia.get_movie(movie_id[2:])  # Remove 'tt' from the ID
        id +=1
        # Check if the movie's genre is in the included genres list
        movie_genres = movie.get('genres', [])
        if any(genre in included_genres for genre in movie_genres):
            # Create URIs for movie and its attributes
            movie_uri = URIRef(ex+'mov'+movie_id[2:])
            
            dict[movie_id[2:]] = movie.get('title')
            # Add triples for movie attributes
            rdf_graph.add((movie_uri, title_uri, Literal(movie_id[2:])))
            rdf_graph.add((movie_uri, published_uri, Literal(movie['year'])))
            
            # Add genre triples
            for genre in movie_genres:
                if genre in included_genres:
                    genre_uri = URIRef(ex+'gen'+genre)
                    #rdf_graph.add((movie_uri, has_genre_uri, genre_uri))
            test_train.append([movie_uri, id, genre_uri])
            # Add director triples
            for director in movie.get('director', []):
                director_name = director.get('name')
                director_id = director.getID()
                dict[director_id] = director_name
                dir_uri = URIRef(ex+'dir'+director_id)
                rdf_graph.add((movie_uri, director_uri, dir_uri))
                rdf_graph.add((dir_uri, dir_mov_uri, movie_uri))
            
            # Add cast triples
            for actor in movie.get('cast', []):
                actor_name = actor.get('name')
                actor_id = actor.getID()
                dict[actor_id] = actor_name
                act_uri = URIRef(ex+'act'+actor_id)
                rdf_graph.add((movie_uri, cast_uri, act_uri))
                rdf_graph.add((act_uri, cast_mov_uri, movie_uri))
                rdf_graph.add((act_uri, collab_uri, dir_uri))
                rdf_graph.add((dir_uri, collab_uri, act_uri))
                rdf_graph.add((genre_uri, prefers_genre, act_uri))
                rdf_graph.add((genre_uri, prefers_genre, dir_uri))
            
    except imdb.IMDbError:
        pass

# Serialize the RDF graph to a file (e.g., Turtle format)
rdf_graph.serialize(destination='filtered_movie_kg_no_genre.rdf', format='turtle')

<Graph identifier=N5cb8308758ca4b36a929560480dcf4b0 (<class 'rdflib.graph.Graph'>)>

In [38]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import os
import gzip
import json
import tqdm

ex = Namespace('http://imdb.org/')
ont = Namespace('http://imdb.org/ontology/')
title_uri = URIRef(ont+'has_title')
has_genre_uri = URIRef(ont+'has_genre')
director_uri = URIRef(ont+'has_director')
dir_mov_uri = URIRef(ont+'directed_by')
cast_uri = URIRef(ont+'has_actor')
cast_mov_uri = URIRef(ont+'acted_in')
published_uri = URIRef(ont+'published_in')
collab_uri = URIRef(ont+'collabs_with')
prefers_genre = URIRef(ont+'preferred_by')

dict = {}
test_train = []
rdf_graph = Graph()
# Genres to include
included_genres = ['Drama', 'Comedy', 'Romance', 'Thriller']
id = 0 

# Define a function to process a single movie
def process_movie(movie_id, included_genres,rdf_graph, id):
    try:
        movie = ia.get_movie(movie_id)  # Remove 'tt' from the ID
        #print(movie)
        id +=1
        # Check if the movie's genre is in the included genres list
        movie_genres = movie.get('genres', [])
        #print(movie_genres)
        #print(included_genres)
        if any(genre in included_genres for genre in movie_genres):
            # Create URIs for movie and its attributes
            movie_uri = URIRef(ex+'mov'+movie_id)
            dict[movie_id] = movie.get('title')
            # Add triples for movie attributes
            rdf_graph.add((movie_uri, title_uri, Literal(movie_id)))
            rdf_graph.add((movie_uri, published_uri, Literal(movie['year'])))
            
            # Add genre triples
            for genre in movie_genres:
                if genre in included_genres:
                    genre_uri = URIRef(ex+'gen'+genre)
                    #rdf_graph.add((movie_uri, has_genre_uri, genre_uri))
            test_train.append([movie_uri, id, genre_uri])
            # Add director triples
            for director in movie.get('director', []):
                director_name = director.get('name')
                director_id = director.getID()
                dict[director_id] = director_name
                dir_uri = URIRef(ex+'dir'+director_id)
                rdf_graph.add((movie_uri, director_uri, dir_uri))
                rdf_graph.add((dir_uri, dir_mov_uri, movie_uri))
            
            # Add cast triples
            for actor in movie.get('cast', []):
                actor_name = actor.get('name')
                actor_id = actor.getID()
                dict[actor_id] = actor_name
                act_uri = URIRef(ex+'act'+actor_id)
                rdf_graph.add((movie_uri, cast_uri, act_uri))
                rdf_graph.add((act_uri, cast_mov_uri, movie_uri))
                rdf_graph.add((act_uri, collab_uri, dir_uri))
                rdf_graph.add((dir_uri, collab_uri, act_uri))
                rdf_graph.add((genre_uri, prefers_genre, act_uri))
                rdf_graph.add((genre_uri, prefers_genre, dir_uri))
            
    except imdb.IMDbError:
        pass

# List of movie IDs
movie_ids = movies['movie'][:1500]
# Create an RDF graph
rdf_graph = Graph()

# Number of concurrent threads/processes
num_threads = 10  # Adjust as needed

# Create a ThreadPoolExecutor
# Create a ProcessPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(process_movie, movie_id[2:], included_genres,rdf_graph, idx + 1) for idx, movie_id in enumerate(movie_ids)]
    for future in concurrent.futures.as_completed(futures):
        pass 
# Serialize the RDF graph to a file (e.g., Turtle format)
rdf_graph.serialize(destination='filtered_movie_kg_parallel.nt', format='turtle')


<Graph identifier=Nae8f207a6bb44e0cb469f0e37fe95bc0 (<class 'rdflib.graph.Graph'>)>

In [65]:
rdf_graph.serialize('data/IMDB/imdb.nt', format='nt')

c:\Users\luisa\anaconda3\envs\aifb\lib\site-packages\rdflib\plugins\serializers\nt.py:35: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


<Graph identifier=Nae8f207a6bb44e0cb469f0e37fe95bc0 (<class 'rdflib.graph.Graph'>)>

In [54]:
train = pd.DataFrame(test_train, columns=['movie', 'id','genre'])#.to_csv('test_train.tsv', index=False)

In [46]:
test_train

[[rdflib.term.URIRef('http://imdb.org/mov0010779'),
  2,
  rdflib.term.URIRef('http://imdb.org/genDrama')],
 [rdflib.term.URIRef('http://imdb.org/mov0010721'),
  3,
  rdflib.term.URIRef('http://imdb.org/genDrama')],
 [rdflib.term.URIRef('http://imdb.org/mov0009753'),
  4,
  rdflib.term.URIRef('http://imdb.org/genDrama')],
 [rdflib.term.URIRef('http://imdb.org/mov0008142'),
  5,
  rdflib.term.URIRef('http://imdb.org/genDrama')],
 [rdflib.term.URIRef('http://imdb.org/mov0016672'),
  6,
  rdflib.term.URIRef('http://imdb.org/genRomance')],
 [rdflib.term.URIRef('http://imdb.org/mov0005300'),
  9,
  rdflib.term.URIRef('http://imdb.org/genDrama')],
 [rdflib.term.URIRef('http://imdb.org/mov0008827'),
  7,
  rdflib.term.URIRef('http://imdb.org/genRomance')],
 [rdflib.term.URIRef('http://imdb.org/mov0015383'),
  11,
  rdflib.term.URIRef('http://imdb.org/genDrama')],
 [rdflib.term.URIRef('http://imdb.org/mov0011890'),
  12,
  rdflib.term.URIRef('http://imdb.org/genRomance')],
 [rdflib.term.URIRef

In [61]:
from sklearn.model_selection import train_test_split
def stratified_train_test_split(train_test):
    train, test = train_test_split(train_test, test_size=0.06, random_state=42, stratify=train_test['genre'])
    return train, test
train, test = stratified_train_test_split(train)
test
train.to_csv('data/IMDB/training_Set.csv', index=False)
test.to_csv('data/IMDB/test_Set.csv', index=False)

In [64]:
tsv_filename = 'movie_kg.tsv'
with open(tsv_filename, 'w') as tsv_file:
    for s, p, o in rdf_graph:
        tsv_file.write(f'{s}\t{p}\t{o}\n')

In [129]:
dict

{'0010779': 'The Thunderbolt',
 '0132324': 'Colin Campbell',
 '0531806': 'Katherine MacDonald',
 '0014894': 'Spottiswoode Aitken',
 '0576762': 'Thomas Meighan',
 '0822477': 'Forrest Stanley',
 '0322284': 'Adda Gleason',
 '0134251': 'Pomeroy Cannon',
 '0365135': 'Mrs. L.C. Harris',
 '0085919': 'Jim Blackwell',
 '0481763': 'Robert Laidlaw',
 '0247483': 'B. Reeves Eason Jr.',
 '0330280': 'James Gordon',
 '0795025': 'Antrim Short',
 '0010721': 'Tarnished Reputations',
 '0085133': 'Herbert Blaché',
 '0349785': 'Alice Guy',
 '0674600': 'Léonce Perret',
 '0144293': 'Dolores Cassinelli',
 '0741182': 'Alan Roscoe',
 '0219013': 'Georges Deneubourg',
 '0123679': 'Ned Burton',
 '0009753': 'The Vanity Pool',
 '0661844': 'Ida May Park',
 '0533712': 'Mary MacLaren',
 '0632160': 'Anna Q. Nilsson',
 '0390366': 'Thomas Holding',
 '0756814': 'Marin Sais',
 '0156346': 'Virginia Chester',
 '0267916': 'Franklyn Farnum',
 '0356228': 'Winter Hall',
 '0115230': 'Frank Brownlee',
 '0548987': 'Willis Marks',
 '0

In [130]:
def st(node):
    """
    Maps an rdflib node to a unique string. We use str(node) for URIs (so they can be matched to the classes) and
    we use .n3() for everything else, so that different nodes don't become unified.

    Source: https://github.com/pbloem/gated-rgcn/blob/1bde7f28af8028f468349b2d760c17d5c908b58b/kgmodels/data.py#L16
    """
    if type(node) == URIRef:
        return str(node)
    else:
        return node.n3()

In [131]:
from collections import Counter
nodes = set()
relations = Counter()
for s, p, o in rdf_graph:
    nodes.add(st(s))
    nodes.add(st(o))

    relations[st(p)] += 1

i2n = list(nodes)

In [132]:
i2n

['http://imdb.org/act0037761',
 'http://imdb.org/act0134251',
 '"1926"^^<http://www.w3.org/2001/XMLSchema#integer>',
 'http://imdb.org/act0449908',
 'http://imdb.org/act0860172',
 'http://imdb.org/act0934232',
 'http://imdb.org/act0149974',
 'http://imdb.org/act0548987',
 'http://imdb.org/act0795025',
 'http://imdb.org/act0385686',
 '"0010779"',
 'http://imdb.org/act0447855',
 'http://imdb.org/act3154246',
 'http://imdb.org/act0329573',
 'http://imdb.org/act0014894',
 'http://imdb.org/dir0002179',
 'http://imdb.org/act0390366',
 'http://imdb.org/act0168916',
 'http://imdb.org/act0381455',
 'http://imdb.org/act0543583',
 'http://imdb.org/act0123679',
 'http://imdb.org/act0914003',
 '"1919"^^<http://www.w3.org/2001/XMLSchema#integer>',
 'http://imdb.org/act0822059',
 'http://imdb.org/act0533712',
 'http://imdb.org/mov0010721',
 'http://imdb.org/act0426718',
 'http://imdb.org/act0125331',
 'http://imdb.org/act0681933',
 'http://imdb.org/act0696390',
 'http://imdb.org/act0322284',
 'http:/

In [121]:
n2i = {n:i for i, n in enumerate(i2n)}
n2i

{'http://imdb.org/act0037761': 0,
 'http://imdb.org/act0134251': 1,
 '"1926"^^<http://www.w3.org/2001/XMLSchema#integer>': 2,
 'http://imdb.org/act0860172': 3,
 'http://imdb.org/act0449908': 4,
 'http://imdb.org/act0934232': 5,
 'http://imdb.org/act0149974': 6,
 'http://imdb.org/act0548987': 7,
 'http://imdb.org/act0795025': 8,
 'http://imdb.org/act0385686': 9,
 '"0010779"': 10,
 'http://imdb.org/act0447855': 11,
 'http://imdb.org/act3154246': 12,
 'http://imdb.org/act0329573': 13,
 'http://imdb.org/act0014894': 14,
 'http://imdb.org/act0390366': 15,
 'http://imdb.org/act0168916': 16,
 'http://imdb.org/dir0002179': 17,
 'http://imdb.org/act0381455': 18,
 'http://imdb.org/act0543583': 19,
 'http://imdb.org/act0914003': 20,
 'http://imdb.org/act0123679': 21,
 '"1919"^^<http://www.w3.org/2001/XMLSchema#integer>': 22,
 'http://imdb.org/act0822059': 23,
 'http://imdb.org/act0533712': 24,
 'http://imdb.org/mov0010721': 25,
 'http://imdb.org/act0426718': 26,
 'http://imdb.org/act0125331': 27,

In [122]:
i2r =list(relations.keys()) 
i2r

['http://imdb.org/ontology/has_actor',
 'http://imdb.org/ontology/acted_in',
 'http://imdb.org/ontology/has_genre',
 'http://imdb.org/ontology/directed_by',
 'http://imdb.org/ontology/has_title',
 'http://imdb.org/ontology/published_in',
 'http://imdb.org/ontology/has_director']

In [89]:
for movie_id in movies['movie'][:10]:
    # Fetch movie information from IMDb
    movie = ia.get_movie(movie_id[2:])
    print(movie)

The Thunderbolt
Tarnished Reputations
The Vanity Pool
The Iron Ring
The Boob
Amarilly of Clothes-Line Alley


KeyboardInterrupt: 

In [79]:
ex[movie_id]


rdflib.term.URIRef('http://imdb.org/tt0015383')

In [62]:
rdf_graph.read('data/IMDB/filtered_movie_kg_parallel.rdf', format='rdf')

AttributeError: 'Graph' object has no attribute 'read'

In [63]:
rdf_graph

<Graph identifier=Nae8f207a6bb44e0cb469f0e37fe95bc0 (<class 'rdflib.graph.Graph'>)>